<a href="https://colab.research.google.com/github/Dvk2002/NLP/blob/main/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=1099572f9819a7c381a849d039892c58b66384d670697dcfca709b123681ebe9
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [2]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 8.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%ls

bin/      datalab/  home/   lib64/  opt/   run/   sys/                tools/
boot/     dev/      lib/    media/  proc/  sbin/  tensorflow-1.15.2/  usr/
content/  etc/      lib32/  mnt/    root/  srv/   tmp/                var/


In [ ]:
cd ../

/


In [4]:

%cd /gdrive/MyDrive/Colab_Notebooks/NLP/L8


/gdrive/MyDrive/Colab_Notebooks/NLP/L8


In [5]:

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")


In [6]:
import numpy as np
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from tensorflow.keras.layers import GlobalMaxPool1D,Bidirectional,LayerNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
df_train['text']

0         alisachachkaич уезжаааааааать ❤ тожена хотеть ...
1         rt galyginvadim ребята девчата кино любовь зав...
2           rt artemklyushin ктоненавидеть пробка ретвит rt
3         rt epupybobv хотеться котлета покиевск запретн...
4         karinekurganova yessboss босапоп есбосан боять...
                                ...                        
181462                     классный новый httptcolejaguxnwu
181463                       видеть человек привет игнорита
181464    julia69styles длинный диагноз вкратце аллергич...
181465    technoslav ух серенький кнопочень телефон дост...
181466    kris1d07 собираться сделатьd навешать фотка ва...
Name: text, Length: 181467, dtype: object

In [8]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
text_corpus_train

array(['alisachachkaич уезжаааааааать ❤ тожена хотеть уезжать',
       'rt galyginvadim ребята девчата кино любовь завтра вотэтолюбовь',
       'rt artemklyushin ктоненавидеть пробка ретвит rt', ...,
       'julia69styles длинный диагноз вкратце аллергический отечь горло осложнение вид фолликулярный ангина',
       'technoslav ух серенький кнопочень телефон достаться',
       'kris1d07 собираться сделатьd навешать фотка вандюхd обязательно выложить фотка нарядишь3'],
      dtype=object)

In [9]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [ ]:
text_corpus_train

array(['alisachachkaич уезжаааааааать ❤ тожена хотеть уезжать',
       'rt galyginvadim ребята девчата кино любовь завтра вотэтолюбовь',
       'rt artemklyushin ктоненавидеть пробка ретвит rt', ...,
       'julia69styles длинный диагноз вкратце аллергический отечь горло осложнение вид фолликулярный ангина',
       'technoslav ух серенький кнопочень телефон достаться',
       'kris1d07 собираться сделатьd навешать фотка вандюхd обязательно выложить фотка нарядишь3'],
      dtype=object)

In [10]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [61]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(128, 3))
model.add(Conv1D(64, 3))
model.add(Conv1D(32, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 27, 100)           25810800  
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 25, 128)           38528     
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 23, 64)            24640     
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 21, 32)            6176      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 32)                0         
_________________________________________________________________
dense_77 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_42 (Dropout)         (None, 64)              

In [62]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
355/355 [==============================] - 143s 399ms/step - loss: 0.5426 - accuracy: 0.7169 - val_loss: 0.5017 - val_accuracy: 0.7485
Epoch 2/10
355/355 [==============================] - 143s 404ms/step - loss: 0.2723 - accuracy: 0.8908 - val_loss: 0.5986 - val_accuracy: 0.7429


In [84]:
class model_k(tf.keras.Model):
    def __init__(self):
        super(model_k, self).__init__()
        self.emb = Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True)
        self.act=Activation("relu")
        # self.norm=LayerNormalization()
        # self.rnn1=SimpleRNN(128,return_sequences=True)
        self.rnn2=SimpleRNN(64,recurrent_dropout=0.2,dropout=0.2,return_sequences=True)
        self.rnn3=SimpleRNN(32,recurrent_dropout=0.2,dropout=0.2,kernel_regularizer='l2')
        # self.pool = GlobalMaxPooling1D()
        self.drop=Dropout(0.2)     
        self.fc1 = Dense(32, activation='relu')
        self.fc2 = Dense(16, activation='relu')
        self.fc3 = Dense(1, activation='sigmoid')

    def call(self, x):

        x = self.emb(x)
        # x= self.rnn1(x)
        x= self.rnn2(x)  
        x= self.rnn3(x)   
        fc_x = self.fc1(self.drop(x))
        fc_x = self.fc2(self.drop(fc_x))
        out = self.fc3(self.drop(fc_x))

        return out

model=model_k()       
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [85]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.1,
                    callbacks=[early_stopping])
model.summary()

Epoch 1/10
355/355 [==============================] - 77s 208ms/step - loss: 0.7016 - accuracy: 0.6650 - val_loss: 0.5272 - val_accuracy: 0.7408
Epoch 2/10
355/355 [==============================] - 73s 206ms/step - loss: 0.3982 - accuracy: 0.8322 - val_loss: 0.5382 - val_accuracy: 0.7435
Model: "model_k_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     multiple                  7743240   
_________________________________________________________________
activation_20 (Activation)   multiple                  0 (unused)
_________________________________________________________________
simple_rnn_10 (SimpleRNN)    multiple                  6080      
_________________________________________________________________
simple_rnn_11 (SimpleRNN)    multiple                  3104      
_________________________________________________________________
dropout_55 (Dropout)         m

In [86]:
class model_k(tf.keras.Model):
    def __init__(self):
        super(model_k, self).__init__()
        self.emb = Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True)
        self.act=Activation("relu")
        # self.norm=LayerNormalization()
        self.rnn1=GRU(128,dropout=0.2,recurrent_dropout=0.2,return_sequences=True)
        self.rnn2=GRU(64,recurrent_dropout=0.2,dropout=0.2,return_sequences=True)
        self.rnn3=GRU(32,recurrent_dropout=0.2,dropout=0.2)
        # self.pool = GlobalMaxPooling1D()
        self.drop=Dropout(0.2)     
        self.fc1 = Dense(32, activation='relu')
        self.fc2 = Dense(16, activation='relu')
        self.fc3 = Dense(1, activation='sigmoid')

    def call(self, x):

        x = self.emb(x)
        # x= self.rnn1(x)
        x= self.rnn2(x)
        x= self.rnn3(x)      
        fc_x = self.fc1(self.drop(x))
        fc_x = self.fc2(self.drop(fc_x))
        out = self.fc3(self.drop(fc_x))

        return out

model=model_k()       
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [87]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.2,
                    callbacks=[early_stopping])
model.summary()

Epoch 1/10
355/355 [==============================] - 223s 609ms/step - loss: 0.5508 - accuracy: 0.7090 - val_loss: 0.4957 - val_accuracy: 0.7535
Epoch 2/10
355/355 [==============================] - 218s 615ms/step - loss: 0.3071 - accuracy: 0.8751 - val_loss: 0.5677 - val_accuracy: 0.7452
Model: "model_k_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_40 (Embedding)     multiple                  25810800  
_________________________________________________________________
activation_21 (Activation)   multiple                  0 (unused)
_________________________________________________________________
gru_48 (GRU)                 multiple                  0 (unused)
_________________________________________________________________
gru_49 (GRU)                 multiple                  31872     
_________________________________________________________________
gru_50 (GRU)                

In [88]:
class model_k(tf.keras.Model):
    def __init__(self):
        super(model_k, self).__init__()
        self.emb = Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True)
        self.act=Activation("relu")
        # self.norm=LayerNormalization()
        self.rnn1=LSTM(128,dropout=0.2,recurrent_dropout=0.2,return_sequences=True)
        self.rnn2=LSTM(64,recurrent_dropout=0.2,dropout=0.2,return_sequences=True)
        self.rnn3=LSTM(32,recurrent_dropout=0.2,dropout=0.2)
        # self.pool = GlobalMaxPooling1D()
        self.drop=Dropout(0.2)     
        self.fc1 = Dense(32, activation='relu')
        self.fc2 = Dense(16, activation='relu')
        self.fc3 = Dense(1, activation='sigmoid')

    def call(self, x):

        x = self.emb(x)
        # x= self.rnn1(x)
        x= self.rnn2(x)
        x= self.rnn3(x)      
        fc_x = self.fc1(self.drop(x))
        fc_x = self.fc2(self.drop(fc_x))
        out = self.fc3(self.drop(fc_x))

        return out

model=model_k()       
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [89]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.2,
                    callbacks=[early_stopping])
model.summary()

Epoch 1/10
355/355 [==============================] - 253s 693ms/step - loss: 0.5504 - accuracy: 0.7117 - val_loss: 0.5004 - val_accuracy: 0.7504
Epoch 2/10
355/355 [==============================] - 244s 687ms/step - loss: 0.3067 - accuracy: 0.8742 - val_loss: 0.6044 - val_accuracy: 0.7413
Model: "model_k_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     multiple                  25810800  
_________________________________________________________________
activation_22 (Activation)   multiple                  0 (unused)
_________________________________________________________________
lstm (LSTM)                  multiple                  0 (unused)
_________________________________________________________________
lstm_1 (LSTM)                multiple                  42240     
_________________________________________________________________
lstm_2 (LSTM)               

In [93]:
class model_k(tf.keras.Model):
    def __init__(self):
        super(model_k, self).__init__()
        self.emb = Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True)
        self.act=Activation("relu")
        # self.norm=LayerNormalization()
        self.rnn2=Bidirectional(GRU(64,return_sequences=True))
        self.rnn3=Bidirectional(GRU(32))
        # self.pool = GlobalMaxPooling1D()
        self.drop=Dropout(0.2)     
        self.fc1 = Dense(32, activation='relu')
        self.fc2 = Dense(16, activation='relu')
        self.fc3 = Dense(1, activation='sigmoid')

    def call(self, x):

        x = self.emb(x)
        # x= self.rnn1(x)
        x= self.rnn2(x)
        x= self.rnn3(x)      
        fc_x = self.fc1(self.drop(x))
        fc_x = self.fc2(self.drop(fc_x))
        out = self.fc3(self.drop(fc_x))

        return out

model=model_k()       
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [94]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.2,
                    callbacks=[early_stopping])
model.summary()

Epoch 1/10
355/355 [==============================] - 270s 718ms/step - loss: 0.5417 - accuracy: 0.7185 - val_loss: 0.4960 - val_accuracy: 0.7528
Epoch 2/10
355/355 [==============================] - 247s 696ms/step - loss: 0.2726 - accuracy: 0.8921 - val_loss: 0.6141 - val_accuracy: 0.7389
Model: "model_k_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_44 (Embedding)     multiple                  25810800  
_________________________________________________________________
activation_25 (Activation)   multiple                  0 (unused)
_________________________________________________________________
bidirectional_2 (Bidirection multiple                  63744     
_________________________________________________________________
bidirectional_3 (Bidirection multiple                  31104     
_________________________________________________________________
dropout_59 (Dropout)        

In [106]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=100,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(64, 3))
# model.add(Conv1D(64, 3))
# model.add(Conv1D(16, 3))
# model.add(GRU(64,return_sequences=True))
model.add(GRU(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optim =tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Adam')

model.compile(
    optimizer=optim, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 27, 100)           25810800  
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 25, 64)            19264     
_________________________________________________________________
gru_64 (GRU)                 (None, 64)                24960     
_________________________________________________________________
dense_143 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_69 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_144 (Dense)            (None, 10)                650       
_________________________________________________________________
dropout_70 (Dropout)         (None, 10)              

In [107]:
early_stopping=EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_valid, y_val),
                    # validation_split=0.1,
                    callbacks=[early_stopping])
model.summary()

Epoch 1/10
355/355 [==============================] - 139s 384ms/step - loss: 0.5405 - accuracy: 0.7190 - val_loss: 0.5081 - val_accuracy: 0.7500
Epoch 2/10
355/355 [==============================] - 134s 377ms/step - loss: 0.2659 - accuracy: 0.8936 - val_loss: 0.6160 - val_accuracy: 0.7384
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_51 (Embedding)     (None, 27, 100)           25810800  
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 25, 64)            19264     
_________________________________________________________________
gru_64 (GRU)                 (None, 64)                24960     
_________________________________________________________________
dense_143 (Dense)            (None, 64)                4160      
_________________________________________________________________
dropout_69 (Dropout)     

В целом, модели с rnn показывают чуть лучше результаты, чем cnn. Что касается комбинированных моделей, то объем тренируемой выборки сравнительно невелик, модель быстро переобучается, и поэтому на ней затруднительно использовать сложные модели. Возможно, есть смысл в качестве токена использовать символ.